# Introduction
We are playing with kaggle's competition that enables us to learn the nuts and bolts of machine learning.In this notebook we are mainly focusing on categorical features.This dataset has 30 features and 1 target.In which 19 features are categorical and 11 of them are numerical features.We will not incorporate any scikit-learn incodindg we will use manual encoding for the preprocessing.

**What is Categorical features and why we need to care bout them?**

Categorical features are the features that includes only a limited and fixed values.Categorical variables represent types of data which may be divided into groups. Examples of categorical variables are race, sex, age group, and educational level.Machine learning models require all input and output variables to be numeric.
This means that if your data contains categorical data, you must encode it to numbers before you can fit and evaluate a model.Encoding is a required pre-processing step when working with categorical data for machine learning algorithms.


# Libraries

In [ ]:
import os
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as ptl
import pandas_profiling as pdp
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score
from sklearn import model_selection
from sklearn import linear_model
from sklearn import metrics
from sklearn import ensemble
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Semi-AutoEDA and data wrangling

In [ ]:
df= pd.read_csv('/kaggle/input/tabular-playground-series-mar-2021/train.csv')
test = pd.read_csv('/kaggle/input/tabular-playground-series-mar-2021/test.csv')
test.head()

In [ ]:
pdp.ProfileReport(df)

In [ ]:
for i in df.columns:
    print (df[i].value_counts())

In [ ]:
for i in test.columns:
    print (test[i].value_counts())

In [ ]:

df['cat0'].value_counts()

In [ ]:
df.shape

# Observations

So we got some insight such as
* Dataset don't have missing values in the datasets.
* There are 4 columns that are even tricky they have combination of 2 letters and 1 letters values.They are cat5,cat7,cat8 and cat10.
* Mainly we are dealing with values in range (A-Z) values so and for our tricky features we also need to a value for missing values.
* We are going to split cat5,cat7,cat8 nad cat10 in two columns each.So that we can extract helpful information out of these columns.

In [ ]:
cat_map ={ 'A':0,
      'B':1,
      'C':2,
      'D':3,
      'E':4,
      'F':5,
      'G':6,
      'H':7,
      'I':8,
      'J':9,
      'K':10,
      'L':11,
      'M':12,
      'N':13,
      'O':14,
      'P':15,
      'Q':16,
      'R':17,
      'S':18,
      'T':19,
      'U':20,
      'V':21,
      'W':22,
      'X':23,
      'Y':24,
      'Z':25,
      '': 26
    
}


In [ ]:
df['cat5_1']=df.cat5.str.slice(start=1)

In [ ]:
df['cat5_2']=df.cat5.str.slice(stop=1)

In [ ]:
df['cat5_1'].head()

In [ ]:
df['cat5_1'].value_counts()

In [ ]:
df['cat5_2'].value_counts()

In [ ]:
df['cat7_1']=df.cat7.str.slice(start=1)
df['cat7_2']=df.cat7.str.slice(stop=1)
df['cat7_1'].value_counts()

In [ ]:
display(df['cat7_2'].value_counts())

In [ ]:
df['cat8_1']=df.cat8.str.slice(start=1)
df['cat8_2']=df.cat8.str.slice(stop=1)
df['cat8_1'].value_counts()

In [ ]:
df['cat8_2'].value_counts()

In [ ]:
df['cat10_1']=df.cat10.str.slice(start=1)
df['cat10_2']=df.cat10.str.slice(stop=1)
df['cat10_1'].value_counts()

In [ ]:
df['cat10_2'].value_counts()

In [ ]:
df.columns

In [ ]:
cat_features=[ 'cat0', 'cat1', 'cat2', 'cat3', 'cat4', 'cat5_1', 'cat5_2', 'cat6', 'cat7_1', 'cat7_2',
       'cat8_1', 'cat8_2', 'cat9', 'cat10_1','cat10_2', 'cat11', 'cat12', 'cat13', 'cat14', 'cat15','cat16', 'cat17', 'cat18']

In [ ]:
for i in cat_features:
    df.loc[:,i]= df[i].map(cat_map)

In [ ]:
for i in cat_features:
    print (df[i].value_counts())

In [ ]:
df['cat8_2'].head()

In [ ]:
df.head()

In [ ]:
drop_col=['cat5','cat7','cat8','cat10']
for i in drop_col:
    df.drop

In [ ]:
df['target'].value_counts()

**As we can see that the target is skewed.And if we use random data to fit in the model we may end up with a biased model.So in order to deal with this problem it is required to incorporate stratified cross-validation technique. So let's go and watch the implementation.**

In [ ]:
df["kfold"] = -1
# the next step is to randomize the rows of the data
df = df.sample(frac=1).reset_index(drop=True)
# fetch targets
y = df['target'].values
# initiate the kfold class from model_selection module
kf = model_selection.StratifiedKFold(n_splits=10)
# fill the new kfold column
for f, (t_, v_) in enumerate(kf.split(X=df, y=y)):
    df.loc[v_, 'kfold'] = f
# save the new csv with kfold column
df.to_csv("train_folds.csv", index=False)

In [ ]:
df.head()

In [ ]:
df_train= df[df.kfold < 9]
df_test= df[df.kfold == 9]

In [ ]:
df.columns

In [ ]:
test.columns

In [ ]:
features=['cat0', 'cat1', 'cat2', 'cat3', 'cat4', 'cat6', 'cat9', 'cat11', 'cat12', 'cat13',
    'cat14', 'cat15',
      'cat16', 'cat17', 'cat18', 'cont0', 'cont1', 'cont2', 'cont3', 'cont4',
       'cont5', 'cont6', 'cont7', 'cont8', 'cont9','cont10',
       'cat5_1', 'cat5_2', 'cat7_1', 'cat7_2', 'cat8_1', 'cat8_2', 'cat10_1',
       'cat10_2']

In [ ]:
X=df_train[features]
y=df_train['target'].values
X_test=df_test[features]
y_test=df_test['target'].values

In [ ]:
print(y)

In [ ]:
model= XGBClassifier(random_state=0,tree_method='gpu_hist',booster= 'gbtree',objective= 'binary:logistic',eval_metric= 'auc',predictor= 'gpu_predictor',learning_rate= 0.055259617966271525, max_depth= 17,gpu_id= 0,subsample= 0.8558107221838249,n_estimators= 552,min_child_weight= 115.85158439509448,colsample_bytree= 0.25704795929046986,reg_alpha= 0.16085318169229096,gamma=0.1378721352250649,max_delta_step=2.2855102006362893,colsample_bylevel= 0.7567272708177056,colsample_bynode= 0.9449337223213273)
model.fit(X,y)

In [ ]:
prediction=model.predict_proba(X_test)[:,1]
rmse=roc_auc_score(y_test, prediction)
print(rmse)

In [ ]:
print(prediction)

In [ ]:
test['cat5_1']=test.cat5.str.slice(start=1)
test['cat5_2']=test.cat5.str.slice(stop=1)

test['cat7_1']=test.cat7.str.slice(start=1)
test['cat7_2']=test.cat7.str.slice(stop=1)

test['cat8_1']=test.cat8.str.slice(start=1)
test['cat8_2']=test.cat8.str.slice(stop=1)

test['cat10_1']=test.cat10.str.slice(start=1)
test['cat10_2']=test.cat10.str.slice(stop=1)
test['cat10_1'].value_counts()

In [ ]:
for i in cat_features:
    test.loc[:,i]= test[i].map(cat_map)

In [ ]:
for i in cat_features:
    print(test[i].value_counts())

In [ ]:
X_sub =test[features]
predictions= model.predict_proba(X_sub)[:,1]

sub_df=pd.read_csv('/kaggle/input/tabular-playground-series-mar-2021/sample_submission.csv')
sub_df.head()
ID=sub_df['id']
submission = pd.DataFrame({'id': ID, 'target': predictions})
submission.to_csv('submissionCats.csv', index = False)

# Till now I done a little bit model tuning so we can expect some more accuracy in roc_auc_score.Please consider upvoting if you like it. Feel free to give suggetions